In [1]:
import reservoirpy as rpy
import pandas as pd
import numpy as np

# rpy.verbosity(0)  # no need to be too verbose here
rpy.set_seed(42)  # make everything reproducible !

In [2]:
from reservoirpy.nodes import Reservoir
# Create the reservoir
reservoir = Reservoir(500, lr=0.5, sr=0.9)

In [3]:
# Load the data
data = pd.read_csv('data.csv', header=0)
# Drop nan values if present
data = data.dropna() if data.isnull().values.any() else data
# Retrieve the needed data columns
X = data[["('temperature', 'mean')","('temperature', 'max')","('temperature', 'min')","('humidite', 'mean')","('humidite', 'max')","('humidite', 'min')","('nuages', 'mean')","('nuages', 'max')","('nuages', 'min')"]]
Y = data[["incidents"]]

In [4]:
# Prepare the data for the reservoir (needs to be a numpy array)
X = np.array(X)
Y = np.array(Y, dtype=np.int32)

print(np.unique(Y))

# Transform Y into one-hot encoding with numpy
Y = np.eye(3)[Y].reshape(-1, 3)
print(Y.shape)

[0 1 2]
(3653, 3)


In [5]:
from sklearn.model_selection import train_test_split
# Split the data into train and test sets
lx, ly = len(X), len(Y)
X_train, X_test, Y_train, Y_test = X[:int(lx*0.8)], X[int(lx*0.8):], Y[:int(ly*0.8)], Y[int(ly*0.8):]

In [6]:
from reservoirpy.nodes import Ridge
ridge = Ridge(ridge=1e-7)
esn_model = reservoir >> ridge

In [7]:
train_states = reservoir.run(X_train)

esn_model = esn_model.fit(X_train, Y_train, warmup=10)

Running Reservoir-0: 100%|██████████| 2922/2922 [00:00<00:00, 4803.03it/s]
Running Model-0: 2922it [00:00, 3336.26it/s]<?, ?it/s]
Running Model-0: 100%|██████████| 1/1 [00:00<00:00,  1.09it/s]


Fitting node Ridge-0...


In [8]:
# Check if the method has initialized all nodes in the Model, and trained the Ridge readout
print(reservoir.is_initialized, ridge.is_initialized, ridge.fitted)

True True True


In [9]:
# Predict 
Y_pred = esn_model.run(X_test)
Y_pred

Running Model-0: 731it [00:00, 1859.69it/s]           


array([[ 9.24375821e-01,  1.77529916e-02,  5.78734378e-02],
       [ 8.84213338e-01,  6.55272566e-02,  5.02604499e-02],
       [ 9.22740544e-01,  4.94378803e-02,  2.78225007e-02],
       ...,
       [ 8.81351425e-01,  7.95132167e-02,  3.91363639e-02],
       [ 8.72873017e-01,  1.05382376e-01,  2.17457421e-02],
       [-3.11566182e+01,  3.23093547e+01, -1.52694645e-01]])

In [10]:
# Predict 
Y_pred_2 = esn_model.run(X_train)
Y_pred_2

Running Model-0: 2922it [00:01, 2726.60it/s]          


array([[-16.07638048,  16.6345679 ,   0.4418289 ],
       [ -7.88578637,   8.4771705 ,   0.4086198 ],
       [ -3.55652427,   4.2974178 ,   0.25910781],
       ...,
       [  0.820183  ,   0.14966888,   0.03014601],
       [  0.81655761,   0.15530593,   0.02813651],
       [  0.92476246,   0.04904759,   0.02618987]])

In [11]:
# compute accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1)))

# compute confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test.argmax(axis=1), Y_pred.argmax(axis=1)))

0.7975376196990424
[[566  27   5]
 [ 86   3   6]
 [ 23   1  14]]


In [12]:
# compute accuracy
print(accuracy_score(Y_train.argmax(axis=1), Y_pred_2.argmax(axis=1)))

# compute confusion matrix
print(confusion_matrix(Y_train.argmax(axis=1), Y_pred_2.argmax(axis=1)))

0.8846680355920602
[[2534   11    0]
 [ 262   35    1]
 [  62    1   16]]
